In [ ]:
from pathlib import Path

In [ ]:
def count_brackets(line):
    cnt = 0
    cnt+=(line.count("(") + line.count("[") + line.count("{"))
    cnt-=(line.count(")") + line.count("]") + line.count("}"))
    return cnt

In [ ]:
def functions_from_kotlin_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except Exception as e:
        print("Error reading the file:", e)
        return []
    
    currFunc = ""
    bracketing = 0
    functions = []
    isInDecl = False #so we count only brackets related to function definitions, not from classes/nested functions etc.
    for x in lines: 
        if "fun " in x and not isInDecl:
            isInDecl = True
        if isInDecl:
            currFunc += x
            bracketing+=count_brackets(x)
            if bracketing==0:
                functions.append(currFunc)
                currFunc = ""
                isInDecl = False
    
    return functions

In [ ]:
def functions_from_dir(directory):
    functions = []
    for file_path in Path(directory).rglob('*.kt'):
        functions += functions_from_kotlin_file(file_path)
    return functions

In [ ]:
functions = functions_from_dir("Kotlin")
functions

In [ ]:
for i in range(0,len(functions)):
    functions[i] = functions[i][functions[i].find("fun"):]

In [ ]:
import json

file_path = "data.json"

with open(file_path, 'w') as json_file:
    for x in functions:
        firstBracket = x.find("{")
        firstEquals = x.find("=")
        if (firstBracket >= 0 and firstBracket <= firstEquals) or (firstBracket >= 0 and firstEquals<0):
            name = x[:firstBracket-1]
            body = x[firstBracket:]
            json_file.write(json.dumps({'Prompt':name, 'Completion':body}) + '\n')
        elif (firstEquals >= 0 and firstBracket >= firstEquals) or (firstBracket < 0 and firstEquals>=0):
            name = x[:firstEquals-1]
            body = x[firstEquals:]
            json_file.write(json.dumps({'Prompt':name, 'Completion':body}) + '\n')
